In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [63]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 12, 3, padding = 1) #input -28 OUtput - 28 RF
        self.batch1 = nn.BatchNorm2d(12) # Batch Normalization 
        self.dropout1 = nn.Dropout(0.10) # Dropout
        self.conv2 = nn.Conv2d(12, 18, 3, padding = 1) #input -28 OUtput - 28 RF
        self.batch2 = nn.BatchNorm2d(18) # Batch Normalization 
        self.pool1 = nn.MaxPool2d(2, 2)   #input -28 OUtput - 14 RF
        self.dropout2 = nn.Dropout(0.25) # Dropout
        self.conv3 = nn.Conv2d(18, 20, 3) #input -14 OUtput - 12 RF
        self.batch3 = nn.BatchNorm2d(20) # Batch Normalization 
        self.dropout3 = nn.Dropout(0.35) # Dropout
        self.conv4 = nn.Conv2d(20, 25, 3) #input -12 OUtput - 10 RF
        self.batch4 = nn.BatchNorm2d(25) # Batch Normalization
        self.pool2 = nn.MaxPool2d(2, 2)   #input -10 OUtput - 5 RF
        self.dropout4 = nn.Dropout(0.25) # Dropout
        self.conv5 = nn.Conv2d(25, 20, 3) #input -5 OUtput - 3 RF
        self.batch5 = nn.BatchNorm2d(20) # Batch Normalization
        self.dropout5 = nn.Dropout(0.35) # Dropout
        self.conv6 = nn.Conv2d(20, 10, 3) #input -3 OUtput - 1


    def forward(self, x):
        x = self.dropout2(self.pool1(self.batch2(F.relu(self.conv2(self.dropout1(self.batch1(F.relu(self.conv1(x)))))))))
        x = self.dropout4(self.pool2(self.batch4(F.relu(self.conv4(self.dropout3(self.batch3(F.relu(self.conv3(x)))))))))
        x = self.dropout5(self.batch5(F.relu(self.conv5(x))))
        x = F.relu(self.conv6(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [64]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
       BatchNorm2d-2           [-1, 12, 28, 28]              24
           Dropout-3           [-1, 12, 28, 28]               0
            Conv2d-4           [-1, 18, 28, 28]           1,962
       BatchNorm2d-5           [-1, 18, 28, 28]              36
         MaxPool2d-6           [-1, 18, 14, 14]               0
           Dropout-7           [-1, 18, 14, 14]               0
            Conv2d-8           [-1, 20, 12, 12]           3,260
       BatchNorm2d-9           [-1, 20, 12, 12]              40
          Dropout-10           [-1, 20, 12, 12]               0
           Conv2d-11           [-1, 25, 10, 10]           4,525
      BatchNorm2d-12           [-1, 25, 10, 10]              50
    

<ipython-input-63-eee253789cce>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [65]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [66]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [67]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-63-eee253789cce>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.11031971126794815 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.28it/s]



Test set: Average loss: 0.0669, Accuracy: 9802/10000 (98%)



loss=0.08926252275705338 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.42it/s]



Test set: Average loss: 0.0504, Accuracy: 9838/10000 (98%)



loss=0.1516779363155365 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.20it/s]


...


Test set: Average loss: 0.0190, Accuracy: 9942/10000 (99%)



loss=0.12562334537506104 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.36it/s]



Test set: Average loss: 0.0184, Accuracy: 9945/10000 (99%)



loss=0.05776094272732735 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.61it/s]



Test set: Average loss: 0.0197, Accuracy: 9942/10000 (99%)

